# Introduction
This data set is taken from [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult)
The goal of this project is to predict whether an individual's annual income exceeds $50K based on various demographic and employment-related features.

The project is divided into three notebooks:
1. Data Cleaning - Prepare the dataset by handling missing values, simplifying categories, and standardizing inputs
2. EDA (Exploratory Data Analysis) – Explore patterns, distributions, and relationships between variables
3. Modeling - Train and evaluate a classification model to predict income level

In [5]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The `adult.data` file does not have column names. So we will make a list of the column names that we will use when we read in the file.

In [6]:
# Create a list of the column names
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'sex',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income'
]

## Data Import and Initial Setup
Read in the `adult.data` file using the  list of column names.

The U.S. Census dataset uses `?` to indicate missing values instead of actual NaN. Replace `?` with NaN so they can be handled properly during cleaning

In [7]:
#read in the adult.data file and add the column names
file_path = '/content/drive/MyDrive/Colab Notebooks/Renewable Energy comp/adult.data'
df = pd.read_csv(file_path, header=None, names=columns, na_values='?', skipinitialspace=True)
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  31978 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [9]:
#check for missing values
df.isnull().sum()

,0
age,0
workclass,1836
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,1843
relationship,0
race,0
sex,0


### Missing Values
The `workclass`, `occupation`, and `native_country` columns contain missing values.
These will be filled using the mode (most frequent) value for each column.
Afterward, we’ll verify that all missing values have been handled.

In [10]:
#fill na values with the mode
df['workclass'] = df['workclass'].fillna(df['workclass'].mode()[0])
df['occupation'] = df['occupation'].fillna(df['occupation'].mode()[0])
df['native_country'] = df['native_country'].fillna('Other')

df.isnull().sum()

,0
age,0
workclass,0
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,0
relationship,0
race,0
sex,0


In [11]:
#check for duplicates
df.duplicated().sum()

np.int64(24)

### Duplicates
There are only 24 duplicated rows in the dataset.
Since this is a small number relative to the full dataset, we’ll drop them without concern for impact.

In [12]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

In [13]:
#Get rid of any white space
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip()

<ipython-input-13-2731528861>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.strip()


In [14]:
# Count empty strings (after stripping whitespace)
for col in df.select_dtypes(include='object').columns:
    has_whitespace = df[col].str.contains(r'^\s|\s$', regex=True).sum()
    print(f"Values with leading/trailing spaces in '{col}': {has_whitespace}")

Values with leading/trailing spaces in 'workclass': 0
Values with leading/trailing spaces in 'education': 0
Values with leading/trailing spaces in 'marital_status': 0
Values with leading/trailing spaces in 'occupation': 0
Values with leading/trailing spaces in 'relationship': 0
Values with leading/trailing spaces in 'race': 0
Values with leading/trailing spaces in 'sex': 0
Values with leading/trailing spaces in 'native_country': 0
Values with leading/trailing spaces in 'income': 0


## Value counts

Many of the categorical columns contain a large number of unique values.
Since this dataset will be used for modeling, we want to consolidate categories where it makes sense.

To guide this process, we’ll use a loop to go through all object-type columns and display their `value_counts(normalize=True)`.
This helps identify low-frequency values that can be grouped together to reduce noise and limit overfitting.

In [15]:
#Print value counts for key object columns to spot typos or outliers.
for col in df.select_dtypes(include='object').columns:
    print(f"\n{col} value counts:")
    print(df[col].value_counts(normalize=True))


workclass value counts:
workclass
Private             0.753266
Self-emp-not-inc    0.078065
Local-gov           0.064327
State-gov           0.039893
Self-emp-inc        0.034299
Federal-gov         0.029505
Without-pay         0.000430
Never-worked        0.000215
Name: proportion, dtype: float64

education value counts:
education
HS-grad         0.322525
Some-college    0.223807
Bachelors       0.164520
Masters         0.052924
Assoc-voc       0.042475
11th            0.036113
Assoc-acdm      0.032793
10th            0.028675
7th-8th         0.019824
Prof-school     0.017703
9th             0.015797
12th            0.013308
Doctorate       0.012693
5th-6th         0.010204
1st-4th         0.005102
Preschool       0.001537
Name: proportion, dtype: float64

marital_status value counts:
marital_status
Married-civ-spouse       0.460092
Never-married            0.327842
Divorced                 0.136491
Separated                0.031503
Widowed                  0.030519
Married-spouse-ab

### 🧹 Category Consolidation Plan
After reviewing the normalized value counts for each categorical column, the following groupings will be applied to simplify the data and reduce the risk of overfitting:

* Workclass

    * Combine `Self-emp-not-inc` and `Self-emp-inc` into `Self-emp`

    * Combine `Without-pay` and `Never-worked` into `No-pay`

* education

    * Group all education levels high school or below into a single category `HS-or-less`

* occupation

    * Combine occupations that make up less than 1% of the data into `Other`
* marital_status

    * Group into `Married`, `Not-Married`, and `Previously-Married` based on relationship history

* native_country

    * Group all countries except `United-States` into `Other`

In [16]:
#Work Class
df['workclass'] = df['workclass'].replace({
    'Self-emp-not-inc': 'Self-emp',
    'Never-worked': 'Without-pay',
    'Self-emp-inc' : 'Self-emp'})
df['workclass'].value_counts(normalize=True)

,proportion
workclass,
Private,0.753266
Self-emp,0.112364
Local-gov,0.064327
State-gov,0.039893
Federal-gov,0.029505
Without-pay,0.000645


In [17]:
#Education
df['education'] = df['education'].replace({
    'HS-grad': 'HS-or-less',
    '11th': 'HS-or-less',
    '10th': 'HS-or-less',
    '7th-8th': 'HS-or-less',
    '9th': 'HS-or-less',
    '12th': 'HS-or-less',
    '5th-6th': 'HS-or-less',
    '1st-4th': 'HS-or-less',
    'Preschool': 'HS-or-less',
    'Assoc-voc': 'Assoc',
    'Assoc-acdm': 'Assoc'
})
df['education'].value_counts(normalize=True)

,proportion
education,
HS-or-less,0.453084
Some-college,0.223807
Bachelors,0.164520
Assoc,0.075268
Masters,0.052924
Prof-school,0.017703
Doctorate,0.012693


In [18]:
# occupation
rare_occupations = df['occupation'].value_counts(normalize=True)
rare_occupations = rare_occupations[rare_occupations < 0.01].index

df['occupation'] = df['occupation'].apply(lambda x: 'Other' if x in rare_occupations else x)
df['occupation'].value_counts(normalize=True)

,proportion
occupation,
Prof-specialty,0.183760
Craft-repair,0.125826
Exec-managerial,0.124935
Adm-clerical,0.115807
Sales,0.112180
Other-service,0.101146
Machine-op-inspct,0.061468
Transport-moving,0.049083
Handlers-cleaners,0.042075


In [19]:
df['marital_status'] = df['marital_status'].replace({
    'Married-civ-spouse': 'Married',
    'Married-spouse-absent': 'Married',
    'Married-AF-spouse': 'Married',
    'Divorced':'Previously-Married',
    'Separated':'Previously-Married',
    'Widowed': 'Previously-Married',
    'Never-married': 'Not-Married'
})
df['marital_status'].value_counts(normalize=True)

,proportion
marital_status,
Married,0.473645
Not-Married,0.327842
Previously-Married,0.198512


In [20]:
#native_country
df['native_country'] = df['native_country'].apply(
    lambda x: x if x == 'United-States' else 'Other'
)
df['native_country'].value_counts(normalize=True)

,proportion
native_country,
United-States,0.895995
Other,0.104005


## Test data set
Now we’ll apply the same transformations to the `adult.test` dataset to ensure it’s cleaned and structured consistently with the training data.
This includes handling missing values, consolidating categories, stripping whitespace, and dropping duplicates if any are present.

In [22]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Renewable Energy comp/adult.test'
df_test = pd.read_csv(file_path, header=None, names=columns, na_values='?', skipinitialspace=True)

In [23]:

def cleaning(data):
    data = data.copy()
    data['workclass'] = data['workclass'].fillna(data['workclass'].mode()[0])
    data['occupation'] = data['occupation'].fillna(data['occupation'].mode()[0])
    data['native_country'] = data['native_country'].fillna('Other')
    data = data.drop_duplicates()
    for col in data.select_dtypes(include='object').columns:
        data[col] = data[col].str.strip()

    data['workclass'] = data['workclass'].replace({
    'Self-emp-not-inc': 'Self-emp',
    'Never-worked': 'Without-pay',
    'Self-emp-inc' : 'Self-emp'})

    data['education'] = data['education'].replace({
    'HS-grad': 'HS-or-less',
    '11th': 'HS-or-less',
    '10th': 'HS-or-less',
    '7th-8th': 'HS-or-less',
    '9th': 'HS-or-less',
    '12th': 'HS-or-less',
    '5th-6th': 'HS-or-less',
    '1st-4th': 'HS-or-less',
    'Preschool': 'HS-or-less',
    'Assoc-voc': 'Assoc',
    'Assoc-acdm': 'Assoc'
    })

    rare_occupations = data['occupation'].value_counts(normalize=True)
    rare_occupations = rare_occupations[rare_occupations < 0.01].index

    data['occupation'] = data['occupation'].apply(lambda x: 'Other' if x in rare_occupations else x)

    data['marital_status'] = data['marital_status'].replace({
    'Married-civ-spouse': 'Married',
    'Married-spouse-absent': 'Married',
    'Married-AF-spouse': 'Married',
    'Divorced':'Previously-Married',
    'Separated':'Previously-Married',
    'Widowed': 'Previously-Married',
    'Never-married': 'Not-Married'
    })

    data['native_country'] = data['native_country'].apply(
    lambda x: x if x == 'United-States' else 'Other'
    )
    return data

### Applying Cleaning to Test Set
The function above applies all the cleaning steps we used on the training dataset. However, the test set may have slight differences, so before writing it to a file, we'll check for any issues like extra whitespace, formatting differences, or unexpected values.

In [24]:
df_test = cleaning(df_test)
df_test.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,|1x3 Cross validator,Private,NaN,NaN,NaN,NaN,Prof-specialty,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN
1,25,Private,226802.0,HS-or-less,7.0,Not-Married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
2,38,Private,89814.0,HS-or-less,9.0,Married,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
3,28,Local-gov,336951.0,Assoc,12.0,Married,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
4,44,Private,160323.0,Some-college,10.0,Married,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.


### Test Set Initial Check
As expected, the test dataset contains some `NaN` values, and the income column includes a trailing `.` character that we’ll need to clean up.

In [25]:
df_test.isna().sum()

,0
age,0
workclass,0
fnlwgt,1
education,1
education_num,1
marital_status,1
occupation,0
relationship,1
race,1
sex,1


In [26]:
df_test = df_test.dropna()
df_test.isna().sum()

,0
age,0
workclass,0
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,0
relationship,0
race,0
sex,0


In [27]:
df_test['income'] = df_test['income'].str.strip('.')
df_test.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
1,25,Private,226802.0,HS-or-less,7.0,Not-Married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
2,38,Private,89814.0,HS-or-less,9.0,Married,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
3,28,Local-gov,336951.0,Assoc,12.0,Married,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
4,44,Private,160323.0,Some-college,10.0,Married,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
5,18,Private,103497.0,Some-college,10.0,Not-Married,Prof-specialty,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K


## Save Cleaned Data
Now that both the training (`adult.data`) and testing (`adult.test`) datasets have been cleaned, we’ll save them as new CSV files. These cleaned versions will be used in the EDA and modeling notebooks.

In [28]:
df.to_csv('clean_adult_train.csv', index=False)
df_test.to_csv('clean_adult_test.csv', index=False)